In [1]:
import torch
import numpy as np
from PIL import Image
from torchvision import transforms as T
from transformers import SegformerForSemanticSegmentation, SegformerConfig
import os

C:\Users\vinib\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.listdir()

['convert_masks.ipynb',
 'count_class_pixels.py',
 'deepglobe-land-cover-2018-DatasetNinja',
 'DeepGlobe_Converted_Dataset',
 'deeplabv3_model.py',
 'models',
 'model_statistics',
 'run_analysis.ipynb',
 'segformer_model.py',
 'segment_and_classify_segformer.py',
 '__pycache__']

In [3]:
model_path = "../b2-ade_30_epochs.pth"
config_path = "models/b2-ade-512-512/config.json"
image_path = 'DeepGlobe_Converted_Dataset/train/119_sat.jpg'
output_mask_path = "segmented_colored_mask.png"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
len(os.listdir('DeepGlobe_Converted_Dataset/train') + os.listdir('DeepGlobe_Converted_Dataset/valid'))/3

803.0

In [4]:
id2label = {
    0: "urban_land",
    1: "agriculture_land",
    2: "rangeland",
    3: "forest_land",
    4: "water",
    5: "barren_land",
    6: "unknown"
}
class_colors = {
    0: (0, 255, 255),
    1: (255, 255, 0),
    2: (255, 0, 255),
    3: (0, 255, 0),
    4: (0, 0, 255),
    5: (255, 255, 255),
    6: (0, 0, 0)
}

In [5]:
config = SegformerConfig.from_json_file(config_path)
model = SegformerForSemanticSegmentation(config)
state_dict = torch.load(model_path, map_location=device)
model.load_state_dict(state_dict)
model.eval().to(device)

FileNotFoundError: [Errno 2] No such file or directory: '../b2-ade_30_epochs.pth'

In [6]:
transform = T.Compose([
    T.Resize((224, 224)),  # Match training resolution
    T.ToTensor(),
    T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

image = Image.open(image_path).convert("RGB")
orig_size = image.size  # (W, H)
input_tensor = transform(image).unsqueeze(0).to(device)

In [7]:
def compute_coverage(mask, num_classes=7):
    total_pixels = mask.size
    coverage = {}
    for class_id in range(num_classes):
        count = np.sum(mask == class_id)
        percent = count / total_pixels * 100
        coverage[id2label[class_id]] = round(percent, 2)
    return coverage

In [8]:
with torch.no_grad():
    outputs = model(pixel_values=input_tensor)
    logits = outputs.logits
    upsampled_logits = torch.nn.functional.interpolate(
        logits,
        size=(orig_size[1], orig_size[0]),  # (H, W)
        mode='bilinear',
        align_corners=False
    )
    pred_mask = upsampled_logits.argmax(dim=1)[0].cpu().numpy()

In [52]:
coverage = compute_coverage(pred_mask)
print("📊 Class-wise Pixel Coverage:")
for label, percent in coverage.items():
    print(f"{label}: {percent:.2f}%")

📊 Class-wise Pixel Coverage:
urban_land: 0.09%
agriculture_land: 25.84%
rangeland: 0.80%
forest_land: 73.08%
water: 0.18%
barren_land: 0.01%
unknown: 0.00%


In [58]:
h, w = pred_mask.shape
color_mask = np.zeros((h, w, 3), dtype=np.uint8)
for class_id, color in class_colors.items():
    color_mask[pred_mask == class_id] = color

Image.fromarray(color_mask).save(output_mask_path)

In [64]:
print("Unique values in mask:", np.unique(pred_mask))


Unique values in mask: [0 1 2 3 4 5]


In [76]:
np.array(Image.open("DeepGlobe_Converted_Dataset/train/119_mask.png"))

array([[2, 2, 2, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=uint8)